In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from runs.models import Run
from run_histos.models import RunHisto

In [3]:
import pandas as pd
import numpy as np
import altair as alt

## Checking apps

In [4]:
runs = Run.objects.all().values()

In [5]:
runs[0]

{'id': 1,
 'run_number': 315257,
 'date': datetime.datetime(2021, 10, 25, 15, 37, 22, 290978, tzinfo=<UTC>)}

In [6]:
run_histos = RunHisto.objects.all().values()

In [7]:
run_histos[0]

{'id': 1,
 'run_id': 1,
 'date': datetime.datetime(2021, 10, 25, 15, 37, 22, 710167, tzinfo=<UTC>),
 'primary_dataset': 'ZeroBias',
 'path': 'TrackParameters/generalTracks/GeneralProperties',
 'title': 'AbsDistanceOfClosestApproachToBS_GenTk',
 'entries': 34430387,
 'mean': 0.4408986607028722,
 'rms': 2.7660848356477787,
 'skewness': 3.1844432366039466,
 'kurtosis': 35.49796721670949}

## Converting to dataframes

In [8]:
df = pd.DataFrame(run_histos)

In [9]:
df.head()

,id,run_id,date,primary_dataset,path,title,entries,mean,rms,skewness,kurtosis
0,1,1,2021-10-25 15:37:22.710167+00:00,ZeroBias,TrackParameters/generalTracks/GeneralProperties,AbsDistanceOfClosestApproachToBS_GenTk,34430387,0.440899,2.766085,3.184443,35.497967
1,2,1,2021-10-25 15:37:22.710269+00:00,ZeroBias,TrackParameters/generalTracks/GeneralProperties,Chi2Prob_GenTk,34430387,0.497623,0.322158,-0.089739,-1.339663
2,3,1,2021-10-25 15:37:22.710310+00:00,ZeroBias,TrackParameters/generalTracks/GeneralProperties,Chi2_GenTk,34430387,18.862586,66.964365,0.046712,-2.909550
3,4,1,2021-10-25 15:37:22.710343+00:00,ZeroBias,TrackParameters/generalTracks/GeneralProperties,Chi2oNDFVsEta_ImpactPoint_GenTk,34421103,-0.103396,1.586303,0.264605,-0.090844
4,5,1,2021-10-25 15:37:22.710387+00:00,ZeroBias,TrackParameters/generalTracks/GeneralProperties,Chi2oNDFVsNHits_ImpactPoint_GenTk,34421103,11.321803,6.708382,38.499120,175.574616


## Filtering

In [10]:
select_title = ['Chi2Prob_GenTk', 'Chi2_GenTk']

In [11]:
df_filtered = df[df['title'].isin(select_title)][['primary_dataset', 'run_id', 'title', 'mean']]

In [12]:
df_filtered

,primary_dataset,run_id,title,mean
1,ZeroBias,1,Chi2Prob_GenTk,0.497623
2,ZeroBias,1,Chi2_GenTk,18.862586
97,EGamma,2,Chi2Prob_GenTk,0.492466
98,EGamma,2,Chi2_GenTk,19.083402
180,EGamma,3,Chi2Prob_GenTk,0.486288
...,...,...,...,...
57534,JetHT,151,Chi2_GenTk,19.614357
57616,JetHT,153,Chi2Prob_GenTk,0.473647
57617,JetHT,153,Chi2_GenTk,19.476241
57699,JetHT,154,Chi2Prob_GenTk,0.442081


In [13]:
df_filtered = df_filtered.pivot(index=['primary_dataset', 'run_id'], columns='title', values='mean').reset_index()

In [14]:
df_filtered

title,primary_dataset,run_id,Chi2Prob_GenTk,Chi2_GenTk
0,EGamma,2,0.492466,19.083402
1,EGamma,3,0.486288,19.146579
2,EGamma,4,0.499791,19.027085
3,EGamma,5,0.489095,19.103232
4,EGamma,6,0.483947,19.190387
...,...,...,...,...
691,JetHT,154,0.442081,20.097355
692,JetHT,542,0.495271,19.129277
693,JetHT,543,0.469204,19.599869
694,JetHT,544,0.478707,19.342482


In [15]:
df_filtered.primary_dataset.unique()

array(['EGamma', 'JetHT', 'ZeroBias'], dtype=object)

## Testing cross-filtering with altair

In [16]:
source = df_filtered

In [17]:
brush_EGamma   = alt.selection(type='interval')
brush_JetHT    = alt.selection(type='interval')
brush_ZeroBias = alt.selection(type='interval')

In [18]:
EGamma = alt.Chart(source[source['primary_dataset']=='EGamma']).mark_point().encode(
    x='Chi2Prob_GenTk:Q',
    y='Chi2_GenTk:Q',
    color=alt.condition(brush_EGamma|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_EGamma
)

In [19]:
JetHT = alt.Chart(source[source['primary_dataset']=='JetHT']).mark_point().encode(
    x='Chi2Prob_GenTk:Q',
    y='Chi2_GenTk:Q',
    color=alt.condition(brush_EGamma|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_JetHT
)

In [20]:
ZeroBias = alt.Chart(source[source['primary_dataset']=='ZeroBias']).mark_point().encode(
    x='Chi2Prob_GenTk:Q',
    y='Chi2_GenTk:Q',
    color=alt.condition(brush_EGamma|brush_JetHT|brush_ZeroBias, 'primary_dataset', alt.value('lightgray'))
).add_selection(
    brush_ZeroBias
)

In [21]:
EGamma & JetHT & ZeroBias

alt.VConcatChart(...)